In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import pprint

# Import the API keys
from config import Client_ID, Client_Secret

In [8]:
#Getting the access token part
igdb_token_base_url = "https://id.twitch.tv/oauth2/token?"
token_data = {
    "client_id":Client_ID,
    "client_secret":Client_Secret,
	"grant_type":"client_credentials"
}
igdb_token_request = requests.post(igdb_token_base_url, data=token_data)
print(igdb_token_request)

<Response [200]>


In [14]:
igdb_token_request.status_code == 200

True

In [15]:
max(True, False)

True

In [9]:
#Print the response
igdb_token_response = igdb_token_request.json()
pprint.pprint(igdb_token_response)

{'access_token': 'ie6wer6g91jy7c5orecx7a79sw5jue',
 'expires_in': 5613714,
 'token_type': 'bearer'}


In [24]:
params = {
    "limit": 500,
    "offset": 0,
    "where": "(category=0 & status=4)",
    "fields": "age_ratings.category, age_ratings.rating, game_modes.name, genres.name, involved_companies.company.name, involved_companies.developer, involved_companies.publisher, language_supports.language.name, multiplayer_modes, name, platforms.name, release_dates.human, release_dates.region, total_rating, total_rating_count"
}
data=""
for key, value in params.items():
    data+=f"{key} {value};\r\n"
print(data)

limit 500;
offset 0;
where (category=0 & status=4);
fields name, genres.name, game_localizations.name, involved_companies.company.name, involved_companies.developer, involved_companies.publisher;



In [23]:
data="'limit 500;\r\noffset 0;\r\nwhere (category=0 & status=4);\r\nfields name, genres.name, game_localizations.name, involved_companies.company.name, involved_companies.developer, involved_companies.publisher;\r\n'"
data

"'limit 500;\r\noffset 0;\r\nwhere (category=0 & status=4);\r\nfields name, genres.name, game_localizations.name, involved_companies.company.name, involved_companies.developer, involved_companies.publisher;\r\n'"

In [27]:
# Store just the access token in a variable.
access_token = igdb_token_response['access_token']

#The base url for getting information about certain games.
games_base_url = "https://api.igdb.com/v4/games"

# Store all the api-key-like values, i.e., Client_ID and access_token, needed to make an api call to get the actual data in a dictionary.
headers = {
    "Client-ID": Client_ID,
    "Authorization": f"Bearer {access_token}"
}

# The query parameters written in a string.
params = {
    "limit": 500,
    "offset": 0,
    "where": "(category=0 & status=4)",
    "fields": "age_ratings.category, age_ratings.rating, game_modes.name, genres.name, involved_companies.company.name, involved_companies.developer, involved_companies.publisher, language_supports.language.name, language_supports.language_support_type.name, multiplayer_modes, name, platforms.name, release_dates.human, release_dates.region, total_rating, total_rating_count"
}

# Placing the parameters in a string, in the required format for the API call to work.
data=""
for key, value in params.items():
    data+=f"{key} {value};\r\n"

games_request = requests.post(games_base_url, headers=headers, data=data)

if games_request.status_code == 200:
    games_response = games_request.json()
    pprint.pprint(games_response)

[{'game_modes': [{'id': 1, 'name': 'Single player'}],
  'genres': [{'id': 32, 'name': 'Indie'}],
  'id': 53308,
  'language_supports': [{'id': 109133,
                         'language': {'id': 7, 'name': 'English'},
                         'language_support_type': {'id': 3,
                                                   'name': 'Interface'}},
                        {'id': 109134,
                         'language': {'id': 7, 'name': 'English'},
                         'language_support_type': {'id': 1, 'name': 'Audio'}}],
  'name': 'Mage Guard',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}],
  'release_dates': [{'human': 'Jul 31, 2017', 'id': 101545, 'region': 8}]},
 {'game_modes': [{'id': 1, 'name': 'Single player'}],
  'genres': [{'id': 12, 'name': 'Role-playing (RPG)'},
             {'id': 15, 'name': 'Strategy'},
             {'id': 31, 'name': 'Adventure'},
             {'id': 32, 'name': 'Indie'}],
  'id': 118128,
  'involved_companies': [{'company': {'id

In [28]:
params['offset']+=500
for key, value in params.items():
    data+=f"{key} {value};\r\n"

games_request = requests.post(games_base_url, headers=headers, data=data)

if games_request.status_code == 200:
    games_response = games_request.json()
    pprint.pprint(games_response)

[{'game_modes': [{'id': 1, 'name': 'Single player'}],
  'genres': [{'id': 12, 'name': 'Role-playing (RPG)'},
             {'id': 31, 'name': 'Adventure'},
             {'id': 32, 'name': 'Indie'}],
  'id': 102192,
  'language_supports': [{'id': 41360,
                         'language': {'id': 7, 'name': 'English'},
                         'language_support_type': {'id': 3,
                                                   'name': 'Interface'}}],
  'name': 'Slime Kingdom',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}],
  'release_dates': [{'human': 'Jun 14, 2018', 'id': 405562, 'region': 8}]},
 {'game_modes': [{'id': 1, 'name': 'Single player'},
                 {'id': 2, 'name': 'Multiplayer'},
                 {'id': 3, 'name': 'Co-operative'}],
  'genres': [{'id': 5, 'name': 'Shooter'},
             {'id': 11, 'name': 'Real Time Strategy (RTS)'},
             {'id': 15, 'name': 'Strategy'},
             {'id': 32, 'name': 'Indie'}],
  'id': 128994,
  'involved_comp